In [1]:
import PyDISORT
import disort
import numpy as np
import matplotlib.pyplot as plt
from math import pi

**PyDISORT**

In [2]:
######################################### PYDISORT ARGUMENTS #######################################

tau_arr = 0.03125
omega_arr = 0.99 # Instead of 1 as that special case has not been implemented
NQuad = 16
Leg_coeffs_all = np.zeros(32)
Leg_coeffs_all[0] = 1
mu0 = 0.1
I0 = pi / mu0
phi0 = 0

# Optional (used)

# Optional (unused)
NLeg=None
NLoops=None
b_pos=0
b_neg=0
only_flux=False
f_arr=0
NT_cor=False
Leg_coeffs_BDRF=np.array([])
s_poly_coeffs=np.array([[]])
use_sparse_NLayers=6

####################################################################################################

**Setup**

In [3]:
# Call PyDISORT
mu_arr, flux_up, flux_down, u = PyDISORT.pydisort(
    tau_arr, omega_arr,
    NQuad,
    Leg_coeffs_all,
    mu0, I0, phi0,
)

In [4]:
# Number of phi grid points
# This selection should ensure that the phi quadrature is at least as accurate as the mu quadrature
Nphi = int((NQuad * pi) // 2) * 2 + 1
phi_arr, full_weights_phi = PyDISORT.subroutines.Clenshaw_Curtis_quad(Nphi)

Ntau = 1000 # Number of tau test points
tau_test_arr = np.random.random(Ntau) * tau_arr

In [5]:
# `mu_arr` was arranged for code efficiency and readability
# For presentation purposes we re-arrange `mu_arr` from smallest to largest.
reorder_mu = np.argsort(mu_arr)

weights_mu = PyDISORT.subroutines.Gauss_Legendre_quad(NQuad//2)[-1] # mu_arr_neg = -mu_arr_pos
full_weights_mu = np.concatenate([weights_mu, weights_mu])
full_weights_mu_RO = full_weights_mu[reorder_mu]
mu_arr_RO = mu_arr[reorder_mu]

**Stamnes' DISORT**

In [6]:
# Stamnes' DISORT arguments
nlyr = 1
nmom = NLeg
nstr = NQuad
numu = NQuad
nphi = Nphi
ntau = Ntau
usrang = True
usrtau = True
ibcnd = 0
onlyfl = False
prnt = np.array([True, False, False, False, False])  # Prints to CMD instead of this notebook
plank = False
lamber = True
deltamplus = False
do_pseudo_sphere = False
dtauc = tau_arr
ssalb = omega_arr
pmom = Leg_coeffs_all.T
temper = np.zeros(nlyr + 1)
wvnmlo = 0
wvnmhi = 0
utau = tau_test_arr
umu0 = mu0
phi0 = phi0
umu = mu_arr_RO
phi = phi_arr
fbeam = I0
fisot = 0
albedo = 0
btemp = 0
ttemp = 0
temis = 0
earth_radius = 6371
h_lyr = np.zeros(nlyr + 1)
rhoq = np.zeros((nstr // 2, nstr + 1, nstr))
rhou = np.zeros((numu, nstr // 2 + 1, nstr))
rho_accurate = np.zeros((numu, nphi))
bemst = np.zeros(nstr // 2)
emust = np.zeros(numu)
accur = 0
header = "Test Problem 3:  Henyey-Greenstein Scattering, g = 0.9 (Compare To Ref. VH2, Table 37)"
rfldir = np.zeros(ntau)
rfldn = np.zeros(ntau)
flup = np.zeros(ntau)
dfdt = np.zeros(ntau)
uavg = np.zeros(ntau)
uu = np.zeros((numu, ntau, nphi))
albmed = np.zeros(numu)
trnmed = np.zeros(numu)

In [7]:
# Run disort, putting DFDT, UAVG, and UU in a, b, and c, respectively
rfldir, rfldn, flup, dfdt, uavg, uu, albmed, trnmed = disort.disort(usrang, usrtau, ibcnd, onlyfl, prnt, plank, lamber, deltamplus, do_pseudo_sphere, dtauc, ssalb,
                        pmom, temper, wvnmlo, wvnmhi, utau, umu0, phi0 * 180/pi, umu, phi * 180/pi, fbeam, fisot, albedo, btemp, ttemp,
                        temis, earth_radius, h_lyr, rhoq, rhou, rho_accurate, bemst, emust, accur, header, rfldir,
                        rfldn, flup, dfdt, uavg, uu, albmed, trnmed)

In [8]:
print("Max pointwise differences")
print()
print("Upward (diffuse) fluxes")

diff_flux_up = np.abs(flup - flux_up(tau_test_arr))
ratio_flux_up = diff_flux_up / np.clip(flup, a_min=1e-6, a_max=None)

print("Difference =", np.max(diff_flux_up))
print("Difference ratio =", np.max(ratio_flux_up))
print()
print("Downward (diffuse) fluxes")

diff_flux_down_diffuse = np.abs((rfldn - flux_down(tau_test_arr)[0]))
ratio_flux_down_diffuse = diff_flux_down_diffuse / np.clip(
    rfldn, a_min=1e-6, a_max=None
)
print("Difference =", np.max(diff_flux_down_diffuse))
print(
    "Difference ratio =",
    np.max(ratio_flux_down_diffuse),
)
print()
print("Direct (downward) fluxes")

diff_flux_down_direct = np.abs(rfldir - flux_down(tau_test_arr)[1])
ratio_flux_down_direct = diff_flux_down_direct / np.clip(rfldir, a_min=1e-6, a_max=None)
print("Difference =", np.max(diff_flux_down_direct))
print(
    "Difference ratio =",
    np.max(ratio_flux_down_direct),
)

Max pointwise differences

Upward (diffuse) fluxes
Difference = 0.0008539820725558079
Difference ratio = 853.9820747119054

Downward (diffuse) fluxes
Difference = 0.0007326570156220846
Difference ratio = 0.005504223527011395

Direct (downward) fluxes
Difference = 0.0016049146127903313
Difference ratio = 0.0006982631519199242


In [9]:
diff = np.abs(uu - u(tau_test_arr, phi_arr)[reorder_mu])
diff_ratio = diff / np.clip(uu, a_min=1e-6, a_max=None)
max_diff_tau_index = np.argmax(np.max(np.max(diff, axis=0), axis=1))
max_ratio_tau_index = np.argmax(np.max(np.max(diff_ratio, axis=0), axis=1))

In [10]:
# Plots will be created for only these values of tau
diff_tau_pt = tau_test_arr[max_diff_tau_index]
ratio_tau_pt = tau_test_arr[max_ratio_tau_index]

In [11]:
print("At tau = " + str(diff_tau_pt))
print("Max pointwise difference =", np.max(diff[:, max_diff_tau_index, :]))
print()
print("At tau = " + str(ratio_tau_pt))
print("Max pointwise difference ratio =", np.max(diff_ratio[:, max_ratio_tau_index, :]))

At tau = 0.031180207316835042
Max pointwise difference = 0.006837105291873369

At tau = 0.031180207316835042
Max pointwise difference ratio = 6837.105309135401


**Does the test pass?**

In [12]:
assert np.max(ratio_flux_up) <= 1e-2 or np.max(diff_flux_up) <= 1e-3
assert np.max(ratio_flux_down_diffuse) <= 1e-2 or np.max(diff_flux_down_diffuse) <= 1e-3
assert np.max(ratio_flux_down_direct) <= 1e-2 or np.max(diff_flux_down_direct) <= 1e-3
assert np.max(diff_ratio) <= 1e-1 or np.max(diff) <= 1e-2

print("The test passes")

The test passes


**Plots (for diagnosis)**

In [13]:
STOP

NameError: name 'STOP' is not defined

**Save outputs**

In [14]:
np.savez(
    "results/test_1b",
    phi_arr=phi_arr,
    tau_test_arr=tau_test_arr,
    uu=uu,
    flup=flup,
    rfldn=rfldn,
    rfldir=rfldir,
)

In [ ]:
MU_ARR, PHI_ARR = np.meshgrid(phi_arr, mu_arr_RO)

In [ ]:
#%matplotlib ipympl

plot = np.log10(diff_ratio[:, max_ratio_tau_index, :])

fig = plt.figure(figsize=(9, 6))
ax = plt.axes(projection="3d")
ax.contourf(MU_ARR, PHI_ARR, plot, 200)
ax.scatter(
    phi0,
    -mu0,
    np.linspace(np.min(plot), np.max(plot), 200),
    marker=".",
    color="red",
    label="direct beam at $\mu$ = "
    + str(-mu0)
    + ", $\phi$ = "
    + str(np.around(phi0, 3)),
)
ax.set_xlabel(r"$\phi$")
ax.set_ylabel(r"$\mu$")
ax.set_zlabel("Log10 of difference ratios")
ax.view_init(30, 140)
plt.title(r"Log10 difference ratios of intensities at $\tau =$" + str(np.around(ratio_tau_pt, 3)))
plt.legend()

In [ ]:
#%matplotlib ipympl

diff_tau_pt = tau_test_arr[max_diff_tau_index]

plot = np.log10(diff[:, max_diff_tau_index, :])

fig = plt.figure(figsize=(9, 6))
ax = plt.axes(projection="3d")
ax.contourf(MU_ARR, PHI_ARR, plot, 200)
ax.scatter(
    phi0,
    -mu0,
    np.linspace(np.min(plot), np.max(plot), 200),
    marker=".",
    color="red",
    label="direct beam at $\mu$ = "
    + str(-mu0)
    + ", $\phi$ = "
    + str(np.around(phi0, 3)),
)
ax.set_xlabel(r"$\phi$")
ax.set_ylabel(r"$\mu$")
ax.set_zlabel("Log10 of difference")
ax.view_init(30, 140)
plt.title(r"Log10 difference in intensities at $\tau =$" + str(np.around(diff_tau_pt, 3)))
plt.legend()